<a href="https://colab.research.google.com/github/cluckey-smith/VinBigData_comp_repo/blob/data_processing/Data_Preprocessing_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import cv2
from google.colab import drive
drive.mount('/content/drive')
import os
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras import datasets, layers, models
import tensorflow as tf
from keras.layers import Convolution2D
from sklearn.model_selection import train_test_split


Mounted at /content/drive


In [58]:
def rgb2gray(rgb):

    r, g, b = rgb[:,:,0], rgb[:,:,1], rgb[:,:,2]
    gray = 0.2989 * r + 0.5870 * g + 0.1140 * b

    return gray

In [63]:
df = pd.read_csv('./train.csv')

In [60]:
train_files = os.listdir('/content/drive/MyDrive/Vin_big_data/train/')

In [89]:
df = df.pivot_table(index='image_id', columns='class_id', values='class_name', aggfunc=lambda x: len(x.unique())).fillna(0)

In [90]:
df.reset_index(inplace=True)


In [ ]:
labels = []
img_name = []
img = []
classes = []

for i,r in df[:1000].iterrows():
  img_name.append(('/content/drive/MyDrive/Vin_big_data/train/') + r['image_id'] + '.png')
  image = load_img(('/content/drive/MyDrive/Vin_big_data/train/') + r['image_id'] + '.png')
  image = img_to_array(image)
  if len(image.shape) > 2:
    image = rgb2gray(image) # Not every image is in grayscale. This is converting them to it
  h,w = image.shape[:2]
  image = image / 255 # Converting our graycale image to values between 0 and 1
  img.append(image)
  classes.append(np.array(df.iloc[i][df.columns[1:]].values))


# Transforming everything to numpy arrays

In [140]:
img = np.array(img)
img_name = np.array(img_name)


In [142]:
classes = np.asarray(classes).astype('float32')

In [143]:
split = train_test_split(img, classes,test_size=0.20, random_state=42)
(trainImages, testImages) = split[:2]
(trainLabels, testLabels) = split[2:4]


# Needed to add a channel

In [144]:
trainImages = trainImages.reshape(len(trainImages), 512, 512, 1)
testImages = testImages.reshape(len(testImages), 512, 512, 1)

# Super duper basic CNN to just play around and get the feet wet

In [145]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.5))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Dropout(0.25))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(len(classes[1]), activation='sigmoid'))

In [146]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 510, 510, 32)      320       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 255, 255, 32)      0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 253, 253, 32)      9248      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 126, 126, 32)      0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 126, 126, 32)      0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 124, 124, 64)      18496     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 62, 62, 64)       

In [147]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model.fit(trainImages, trainLabels, epochs=10, 
                    validation_data=(testImages, testLabels))

Epoch 1/10
8/8 [==============================] - 93s 11s/step - loss: 0.4097 - accuracy: 0.3500 - val_loss: 0.4197 - val_accuracy: 0.5833
Epoch 2/10
8/8 [==============================] - 83s 10s/step - loss: 0.2885 - accuracy: 0.6625 - val_loss: 0.3991 - val_accuracy: 0.6167
Epoch 3/10
8/8 [==============================] - 83s 10s/step - loss: 0.2773 - accuracy: 0.6792 - val_loss: 0.3543 - val_accuracy: 0.6167
Epoch 4/10
8/8 [==============================] - 82s 10s/step - loss: 0.2672 - accuracy: 0.6667 - val_loss: 0.3075 - val_accuracy: 0.6167
Epoch 5/10
8/8 [==============================] - 83s 10s/step - loss: 0.2609 - accuracy: 0.7000 - val_loss: 0.3052 - val_accuracy: 0.6167
Epoch 6/10
3/8 [==========>...................] - ETA: 51s - loss: 0.2506 - accuracy: 0.6979 

KeyboardInterrupt: ignored

In [ ]:
model.save('/content/drive/MyDrive/Vin_big_data/model2')